In [48]:
device = 'cuda' if torch.cuda.is_available else "cpu"
torch.manual_seed(777)
if device=="cuda":
    torch.cuda.manual_seed_all(777)

In [49]:
learning_rate=1e-3
training_epochs=15
batch_size=100

In [50]:
mnist_train=dsets.MNIST(root=os.path.join(img_path,"MNIST_data/"),
               train=True,
               transform=transforms.ToTensor(),
               download=True)
mnist_test=dsets.MNIST(root=os.path.join(img_path,"MNIST_data/"),
               train=False,
               transform=transforms.ToTensor(),
               download=True)

In [51]:
data_loader=torch.utils.data.DataLoader(dataset=mnist_train,
                                 batch_size=batch_size,
                                 shuffle=True,
                                 drop_last=True)

In [52]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer1=nn.Sequential(
        nn.Conv2d(1,32,3,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2))
        self.layer2=nn.Sequential(
        nn.Conv2d(32,64,3,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2))
        self.fc=nn.Linear(7*7*64,10,bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)
    
    def forward(self,x):
        out=self.layer1(x)
        out=self.layer2(out)
        out=out.view(out.size(0),-1)
        out=self.fc(out)
        return out

In [53]:
model=CNN().to(device)

In [54]:
criterion=nn.CrossEntropyLoss().to(device)
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [55]:
total_batch=len(data_loader)
for epoch in range(training_epochs):
    avg_cost=0
    for X,y in data_loader:
        X=X.to(device)
        y=y.to(device)
        
        optimizer.zero_grad()
        hypothesis=model(X)
        cost=criterion(hypothesis,y)
        cost.backward()
        optimizer.step()
        
        avg_cost+=cost/total_batch
    print(epoch,"cost",avg_cost)
print("learning complete")
        

0 cost tensor(0.2238, device='cuda:0', grad_fn=<AddBackward0>)
1 cost tensor(0.0621, device='cuda:0', grad_fn=<AddBackward0>)
2 cost tensor(0.0448, device='cuda:0', grad_fn=<AddBackward0>)
3 cost tensor(0.0355, device='cuda:0', grad_fn=<AddBackward0>)
4 cost tensor(0.0290, device='cuda:0', grad_fn=<AddBackward0>)
5 cost tensor(0.0249, device='cuda:0', grad_fn=<AddBackward0>)
6 cost tensor(0.0209, device='cuda:0', grad_fn=<AddBackward0>)
7 cost tensor(0.0180, device='cuda:0', grad_fn=<AddBackward0>)
8 cost tensor(0.0150, device='cuda:0', grad_fn=<AddBackward0>)
9 cost tensor(0.0125, device='cuda:0', grad_fn=<AddBackward0>)
10 cost tensor(0.0103, device='cuda:0', grad_fn=<AddBackward0>)
11 cost tensor(0.0097, device='cuda:0', grad_fn=<AddBackward0>)
12 cost tensor(0.0085, device='cuda:0', grad_fn=<AddBackward0>)
13 cost tensor(0.0075, device='cuda:0', grad_fn=<AddBackward0>)
14 cost tensor(0.0066, device='cuda:0', grad_fn=<AddBackward0>)
learning complete


In [66]:
with torch.no_grad():
    X_test=mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    y_test=mnist_test.test_labels.to(device)
    pred=model(X_test)
    correct_pred=torch.argmax(pred,1)==y_test
    accuracy=correct_pred.float().mean()
    print(accuracy)
    
    

tensor(0.9862, device='cuda:0')


In [62]:
len(mnist_test.test_data.size())

3